# UMPS homework 1 template

Let's use the following convention for numbering legs:
```
 1--A--3
    |
    2
```

In [1]:
using TensorOperations
using LinearMaps

In [2]:
"""
    rand_UMPS(d, D; keep_it_real=true)

Return a random three-valent tensor A, that defines a uniform MPS (UMPS).
The bond dimension of the physical leg should be d, and the bond dimension
of the two "virtual" legs (the horizontal ones) should be D.
keep_it_real is keyword argument, for whether the matrix should be real or
complex.

This means you can call
`rand_UMPS(2, 9)`
or
`rand_UMPS(2, 9; keep_it_real=true)`
and they both give a you real A, but you can also call
`rand_UMPS(2, 9; keep_it_real=false)`
to get a complex A.
"""
function rand_UMPS(d, D; keep_it_real=true)
    shp = (D, d, D)
    if keep_it_real
        A = randn(shp)
    else
        A_real = randn(shp)
        A_imag = randn(shp)
        A = complex.(A_real, A_imag) / sqrt(2)
    end
    return A
end

rand_UMPS

In [3]:
"""
    tm(A)

Return the transfer matrix of A:
 i1--A---j1
     |  
 i2--A*--j2
"""
function tm(A)
    @tensor T[i1,i2,j1,j2] := A[i1,p,j1]*conj(A)[i2,p,j2]
end

tm

In [4]:
function eig_and_trunc(T, nev)
    S, U = eig(T)
    perm = sortperm(S; by=abs, rev=true)
    S = S[perm]
    U = U[:, perm]
    S = S[1:nev]
    U = U[:, 1:nev]
    return S, U
end

"""
    tm_eigs(A, dirn, nev)

Return some of the eigenvalues and vectors of the transfer matrix of A.
dirn should be "L", "R" or "BOTH", and determines which eigenvectors to return.
nev is the number of eigenpairs to return (starting with the eigenvalues with
largest magnitude).
"""
function tm_eigs_dense(A, dirn, nev)
    T = tm(A)
    D = size(T, 1)
    T = reshape(T, (D^2, D^2))
    nev = min(nev, D^2)
    
    result = ()
    if dirn == "R" || dirn == "BOTH"
        SR, UR = eig_and_trunc(T, nev)
        UR = [reshape(UR[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SR, UR)
    end
    if dirn == "L" || dirn == "BOTH"
        SL, UL = eig_and_trunc(T', nev)
        UL = [reshape(UL[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SL, UL)
    end
    return result
end

tm_eigs_dense

In [5]:
"""
    normalize!(A)

Normalize the UMPS defined by A. This is done by dividing A by the square root of
the dominant (largest magnitude) eigenvalue of the MPS transfer matrix.
"""
function normalize!(A)
    S, U = tm_eigs_dense(A, "R", 1)
    S1 = S[1]
    A ./= sqrt(S1)
    return A
end

normalize!

In [6]:
let
    d = 2
    D = 10
    A = rand_UMPS(d, D; keep_it_real=false)

    SR, UR, SL, UL = tm_eigs_dense(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
    normalize!(A)
    SR, UR, SL, UL = tm_eigs_dense(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
end

SR[1] = 20.525050253965077 - 1.2434497875801753e-14im
SL[1] = 20.52505025396496 - 8.386786882106903e-15im
SR[1] = 1.0000000000000029 - 2.220446049250313e-16im
SL[1] = 0.9999999999999956 - 8.875991479296312e-16im


0.9999999999999956 - 8.875991479296312e-16im

### Hurray!
Now let's get smart about diagonalizing that MPS transfer matrix.

In [7]:
"""
    tm_l(A, x)

Return y, where
/------   /------A--
|       = |      |  
\- y* -   \- x* -A*-
"""
function tm_l(A, x)
    @tensor y[i, j] := (x[a, b] * A[b, p, j]) * conj(A[a, p, i])
    return y
end


"""
    tm_r(A, x)

Return y, where
-- y -\   --A-- x -\
      | =   |      |
------/   --A*-----/
"""
function tm_r(A, x)
    @tensor y[i, j] := A[i, p, a] * (conj(A[j, p, b]) * x[a, b])
    return y
end


tm_r

In [8]:
function tm_eigs_sparse(A, dirn, nev)
    if dirn == "BOTH"
        SR, UR = tm_eigs_sparse(A, "R", nev)
        SL, UL = tm_eigs_sparse(A, "L", nev)
        return SR, UR, SL, UL
    else
        D = size(A, 1)
        x = zeros(eltype(A), (D, D))
        if dirn == "L"
            f = v -> vec(tm_l(A, copy!(x, v)))
        else
            f = v -> vec(tm_r(A, copy!(x, v)))
        end

        fmap = LinearMap{eltype(A)}(f, D^2)
        S, U, nconv, niter, nmult, resid = eigs(fmap, nev=nev, which=:LM, ritzvec=true)
        U = [reshape(U[:,i], (D, D)) for i in 1:size(U, 2)]

        return S, U
    end
end



tm_eigs_sparse (generic function with 1 method)

In [9]:
let
    d = 2
    D = 40
    A = rand_UMPS(d, D; keep_it_real=false)

    nev = 2
    @time SR_sparse, UR_sparse = tm_eigs_sparse(A, "R", nev)
    @time SR_dense, UR_dense = tm_eigs_dense(A, "R", nev)
    println("\nComparison of eigenvalues:")
    println(SR_dense)
    println(SR_sparse)
    println("\nComparison of eigenvectors:")
    println(UR_dense[1][1:6])
    println(UR_sparse[1][1:6])
    println("\nComparison of abs.(eigenvectors):")
    println(abs.(UR_dense[1])[1:6])
    println(abs.(UR_sparse[1])[1:6])
end

  3.475528 seconds (2.32 M allocations: 292.103 MiB, 1.69% gc time)
 21.932633 seconds (194 allocations: 196.494 MiB, 0.53% gc time)

Comparison of eigenvalues:
Complex{Float64}[80.8867+1.24345e-14im, -6.41722-58.6043im]
Complex{Float64}[80.8867+1.96076e-14im, -6.41722-58.6043im]

Comparison of eigenvectors:
Complex{Float64}[0.13274+3.1225e-17im, 0.013985+0.00483383im, -0.00543919+0.0201979im, 0.0190075+0.0029064im, -0.000825505+0.00591032im, 0.0106425+0.00524406im]
Complex{Float64}[-0.112287+0.0707916im, -0.0144081+0.00336933im, -0.00617067-0.0199866im, -0.0176289+0.00767835im, -0.00245373-0.00543991im, -0.0117994+0.0012397im]

Comparison of abs.(eigenvectors):
[0.13274, 0.0147968, 0.0209175, 0.0192285, 0.0059677, 0.0118643]
[0.13274, 0.0147968, 0.0209175, 0.0192285, 0.0059677, 0.0118643]


In [10]:
function tm_eigs(A, dirn, nev; max_dense_D=10)
    D = size(A, 1)
    if D <= max_dense_D || nev >= D^2
        return tm_eigs_dense(A, dirn, nev)
    else
        return tm_eigs_sparse(A, dirn, nev)
    end
end

tm_eigs (generic function with 1 method)

## Onwards to new things: Computing expectation values

The expectation value of an operator $O$, for a generic quantum state $|\psi\rangle$, is $\frac{\langle \psi | O | \psi \rangle}{\langle \psi | \psi \rangle}$. Since we have now normalized our MPS state, we can ignore the denominator, and concentrate on $\langle \psi | O | \psi \rangle$.

For a UMPS state $| \text{UMPS}(A) \rangle$ and a single site operator $O$ this expectation value can be computed as follows:

<img src="fig/expect_local.svg">

We have made use here of the fact that we're always living at the limit $N \to \infty$, and thus $T^N = |r_1\rangle \lambda_1^N \langle l_1|$, where $\lambda_1$ is the dominant eigenvalue, which we further know is $1$.

Similarly, the expectation value of the tensor product of two one-site operators, i.e. a two-point correlator, can be computed as follows:

<img src="fig/correlator_twopoint.svg">

Here $O_1^{(0)}$ is located on site $0$ and $O_1^{(m)}$ is located on site $m$ (note that everything is translation invariant, so this is fully general).

This seems simple enough! First, we need to get the dominant left and right eigenvectors of $T$, $|r_1\rangle$ and $\langle l_1|$. That we are already doing when implementing the normalization. Second, we need a function that applies $T_O$ to a vector in the $D^2$ dimensional virtual vector space, i.e., a function like `tm_l` and `tm_r`, but now with an operator $O$ included in the middle. (Throughout all this, I recommend thinking of the vectors in the $D^2$ dimensional space as $D \times D$ matrices, as for instance is done in the functions `tm_r` and `tm_l`.) Using such a function, we can first evaluate for instance $T_{O_2} | r_1 \rangle = |v \rangle$, then multiply $|v \rangle$ by $T$ $m-1$ times, and finally once by $T_{O_1}$. At the end we are left with something like $\langle l_1| v \rangle$, which is just an inner product in the $D^2$ dimensional vector space:

<img src="fig/D2_inner_product.svg">

Of course, we could also start the contraction from the left instead.

In light of this, your homework would naturally be to implement functions with the following signatures:

`tm_r_op(A, O, x)`: Transfer matrix with an operator, multiplied from the right.

`tm_l_op(A, O, x)`: Transfer matrix with an operator, multiplied from the left.

`expect_local(A, O)`: Expectation value of $O$.

`correlator_twopoint(A, O1, O2, m)`: Two-point correlators between two operators, distance $m$ apart. You should compute the connected two-point correlators, i.e., $\langle O_1^{(0)} O_2^{(m)} \rangle - \langle O_1 \rangle \langle O_2 \rangle$. In addition, since on the way to computing the two-point correlator for distance $m$, you end computing all the two-point correlators for distance $i \leq m$, you should make the function return all of them, because why not.

However, before you jump into writing code, there's a slight additional complication. Since we often want to decompose $T$ as $T = \sum_{i=1}^{D^2} |r_i \rangle \lambda_i \langle l_i|$, we need to make sure that not only are the $|r_i \rangle$ and $\langle l_i|$ we have eigenvectors of $T$, but also that they are normalized so that $\langle l_i| r_j \rangle = \delta_{ij}$. If we do not impose this, there's no guarantee that the eigenvectors returned by `eigs` or `eig` have this property: $\langle l_i| r_j \rangle$ will be diagonal, but the values on the diagonal are not necessarily $1$, unless we scale the eigenvectors ourselves to force this. We'll only really ever need $|r_1 \rangle$ and $\langle l_1|$, i.e., the dominant ones, and we are already computing them as part of the normalization process. Thus it makes sense to modify the `normalize!` function so that, in addition to normalizing the MPS, it also normalizes the dominant eigenvectors, and returns them. I've written such a modified `normalize!` function below.

In [11]:
"""
    normalize!(A)

Normalize the UMPS defined by A, and return the dominant left and right
eigenvectors l and r of its transfer matrix, normalized so that l'*r = 1.
"""
function normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    S1 = SR[1]
    A ./= sqrt(S1)  # Normalizing the state.
    
    # Normalizing the eigenvectors
    l = UL[1]
    r = UR[1]  
    #We need this to be 1
    n = vec(l)'*vec(r)
    abs_n = abs(n)
    phase_n = abs_n/n
    sfac = 1.0/sqrt(abs_n)
    l .*= sfac/phase_n
    r .*= sfac
    return l, r
end

normalize!

The above cell should have overridden the previous definition of `normalize!` that we had.

Let's just quickly confirm that this works:

In [14]:
let
    d = 2
    D = 20
    A = rand_UMPS(d, D; keep_it_real=false)
    
    l, r = normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    println(SR[1] ≈ 1.0)
    println(SL[1] ≈ 1.0)
    
    println(l ≈ tm_l(A, l))
    println(r ≈ tm_r(A, r))
    println(vec(l)'*vec(r) ≈ 1.0)
end

true
true
true
true
true


(By the way, I'm putting some of the code in these `let`-`end` blocks. That's to prevent names that I define in these blocks from polluting the global namespace: If I define `d = 2` within the `let`-`end` block, `d` will still be an unassigned name outside the block. Just a bit of basic code hygiene, when working with notebooks.)

Armed with this new `normalize!` function, we are all set to implement computing expectation values. The natural way to proceed is to call `normalize!` once on an MPS after you create or modify it, and store the dominant eigenvectors that `normalize!` returns (I call them `l` and `r` in the code). These eigenvectors should then be passed to the functions that need them, so that they don't need to recompute them.

Below I've made empty template functions to compute the expectation values we were talking about above. Your job is to fill in the blanks.

## Homework 2 template

In [15]:
"""
    expect_local(A, O, l, r)

Return the expectation value of the one-site operator O for the (normalized)
UMPS state defined by the tensor A. l and r should be the dominant
eigenvectors of the MPS transfer matrix of A.
"""
function expect_local(A, O, l, r)
    # ???
end

expect_local

In [17]:
"""
    correlator_twopoint(A, O1, O2, m, l, r)

Return the (connected) two-point correlator of operators O1 and O2 for the
state UMPS(A), when O1 and O2 are i sites apart, where i ranges from 1 to m. In
other words, return <O1^(0) O2^(i)> - <O1> <O2>, for all i = 1,...,m, where the
expectation values are with respect to the state |UMPS(A)>. l and r should be
the dominant eigenvectors of the MPS transfer matrix of A.
"""
function correlator_twopoint(A, O1, O2, m, l, r)
    # ???
end

correlator_twopoint

In [18]:
"""
    tm_l_op(A, O, x)

Return y, where
/------   /------A--
|         |      |  
|       = |      O  
|         |      |  
\- y* -   \- x* -A*-
"""
function tm_l_op(A, O, x)
    # ???
end


"""
    tm_r_op(A, O, x)

Return y, where
-- y -\   --A-- x -\
      |     |      |
      | =   O      |
      |     |      |
------/   --A*-----/t
"""
function tm_r_op(A, O, x)
    # ???
end


tm_r_op

## Ha, you thought you were done!

Let's take a look at the expression for the connected two-point correlator:
$\langle O_1^{(0)} O_2^{(m)} \rangle - \langle O_1 \rangle \langle O_2 \rangle
= \langle l_1 | T_{O_1}  T^{m-1} T_{O_2} | r_1 \rangle$<br>
Consider what happens when $m$ is large. Then we can also decompose<br>
$T^{m-1} = \sum_{i=1}^{D^2} |r_i \rangle \lambda_i^{m-1} \langle l_i|
\approx |r_1 \rangle \langle l_1|$.<br>
Plugging this into the above equation,
$\langle O_1^{(0)} O_2^{(m)} \rangle - \langle O_1 \rangle \langle O_2 \rangle
\approx \langle l_1 | T_{O_1} |r_1 \rangle \langle l_1| T_{O_2} |r_1 \rangle - \langle O_1 \rangle \langle O_2 \rangle = \langle O_1 \rangle \langle O_2 \rangle - \langle O_1 \rangle \langle O_2 \rangle = 0.$<br>
Well duh, we've discovered that at large distances the connected correlation function goes to zero. We can get something more interesting if we also take into account the second largest eigenvalue of $T$:
$T^{m-1} \approx |r_1 \rangle \langle l_1| + |r_2 \rangle \lambda_2^{m-1} \langle l_2|$<br>
$\Rightarrow
\langle O_1^{(0)} O_2^{(m)} \rangle - \langle O_1 \rangle \langle O_2 \rangle
\approx \langle l_1 | T_{O_1} |r_2 \rangle \lambda_2^{m-1} \langle l_2| T_{O_2} |r_1 \rangle
= C(A, O_1, O_2) ~ \phi(A, m) ~ |\lambda_2|^m.$<br>
Here $C$ is the part with the inner products that doesn't depend on $m$ at all, and $\phi$ is some phase that may depend on $m$ (if $O_1$ and $O_2$ are Hermitian operators $\phi$ is just a sign). The magnitude of the correlation function depends on the distance as
$|\lambda_2|^m$, which we rewrite as $e^{m \ln |\lambda_2|} = e^{- \frac{m}{\xi}}$, with $\xi = -\frac{1}{\ln |\lambda_2|}$.

In other words, for an MPS (for *any* MPS!), at large distances, two-point correlators always decay exponentially, with a *correlation length* $\xi = -\frac{1}{\ln |\lambda_2|}$, where $\lambda_2$ is the second largest eigenvalue of the MPS transfer matrix (by absolute value).

Now that's a good thing to know! For one, the correlation length of a state is sometimes of physical interest in itself. In addition, this gives a very useful characterization of the kinds of states MPS can represent: If you want to represent a state as an MPS, the state better have exponential decay of correlators, or you're in trouble. As you can probably guess, low-energy states of local Hamiltonians typically have this behavior (except for gapless Hamiltonians).

Given this, as a final piece, you should try to implement the function below that computes the correlation length. You could then even compare the two-point correlators you are computing numerically, to check that they really do decay like $e^{- \frac{m}{\xi}}$. A good idea would be for instance to plot the numerical data against $e^{- \frac{m}{\xi}}$. We can look into some basic plotting next time, but if you want to go ahead, you can try starting here: https://julialang.org/downloads/plotting.html

In [19]:
"""
    correlation_length(A)

Return the correlation length ξ of the (normalized) UMPS defined by A.
"""
function correlation_length(A)
    # ???
end

correlation_length